In [122]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [123]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (178977, 28, 28) (178977,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [116]:
def labelarray(X):
    try:
        A = np.zeros((X.shape[0],10))

        for i in range (0,X.shape[0]):
            A[i,X[i]]=1
    except Exception as e:
        print('labelarray error:',e);
        
    return A

In [117]:
# 1-hot enodes the labels

train_labels = labelarray(train_labels)
valid_labels = labelarray(valid_labels)
test_labels = labelarray(test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (178977, 28, 28) (178977, 10)
Validation set (10000, 28, 28) (10000, 10)
Test set (10000, 28, 28) (10000, 10)


In [124]:
# 1-hot encodes the labels and tranforms each image into a prism, need depth for covnets.
# If labels are already 1-hot encoded it will give them a depth of 1 as well. Only run once!

image_size = 28
num_labels = 10
num_channels = 1 # grayscale depth is 1

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (178977, 28, 28, 1) (178977, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [125]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [126]:
# Initialize the Tensorflow model. Convolution output will have a depth of 16. There will be 2 convolution 
# layers followed by a fully connected hidden layer with 64 nodes. The convolution layers will convolve over a 
# 4*4 patch. Convolution layers will have same padding. for now the loss function won't aim to reduce single  
# bias.

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    pool = tf.nn.max_pool(data,[1,2,2,1],[1,2,2,1],padding='SAME')
    conv = tf.nn.conv2d(pool, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # Optimizer.
  learningrate = tf.placeholder(tf.float32)
  optimizer = tf.train.GradientDescentOptimizer(learningrate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [127]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, learningrate : 3*(1-np.exp(-l/(1+l)))*np.sqrt(1001-step)/1000}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.025176
Minibatch accuracy: 6.2%
Validation accuracy: 9.3%
Minibatch loss at step 50: 1.889948
Minibatch accuracy: 25.0%
Validation accuracy: 44.6%
Minibatch loss at step 100: 1.178835
Minibatch accuracy: 68.8%
Validation accuracy: 74.0%
Minibatch loss at step 150: 1.175274
Minibatch accuracy: 68.8%
Validation accuracy: 75.0%
Minibatch loss at step 200: 0.580698
Minibatch accuracy: 87.5%
Validation accuracy: 77.1%
Minibatch loss at step 250: 1.088492
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.967046
Minibatch accuracy: 62.5%
Validation accuracy: 79.4%
Minibatch loss at step 350: 0.769289
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.675498
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 450: 1.129809
Minibatch accuracy: 62.5%
Validation accuracy: 81.0%
Minibatch loss at step 500: 0.387366
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Min